# Market Basket Analysis
GitHub location: https://github.com/LarsTinnefeld/olist_ecom_analysis.git

**A project of the Olist ecommerce business analysis.**

<img src="https://i2.wp.com/dataneophyte.com/wp-content/uploads/2019/12/Logo-01.png" width="400" height="300">


## Questions to answer
...

## Table of Contents

I. [Data Import and Wrangling](#data)<br>
II. [Exploratory Date Analysis](#eda)<br>
III. [Market Basket Analysis](#affinity)<br>

---
## <a class="anchor" id="data">I. Data Import and Wrangling</a>

### 1. Libraries

In [2]:
import pandas as pd
import numpy as np
from apyori import apriori
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime as dt
%matplotlib inline
sns.set_style("whitegrid")

### 2. Importing order data
Part of the data was inherited from the initial Olist data analysis.

General data structure:

<img src="https://i.imgur.com/HRhd2Y0.png" width="700" height="450">

We will import the already cleanded dataset from the shared dataset of the Olist business analysis.

<img src="https://github.com/LarsTinnefeld/olist_ecom_analysis/blob/main/Olist-Analysis_1_New_tables.PNG?raw=true" width="700" height="400">

---
## <a class="anchor" id="affinity">III. Market Basket Analysis</a>

Questions to answer:

** Can we predict buying behaviour between articles in one order (association)? **

I will try to answer these questions with an Affinity Analysis (Market Basket Analysis) by using the Apyori library. Basic formulas:

### 1. Basics

How likely SKU B is purchased if SKU A is purchased: $$ Confidence(A\Rightarrow B) = \frac{frq(A, B)}{frq(A)}) $$

How popular is SKU A: $$ Support(A\Rightarrow B) = \frac{frq(A, B)}{N}) $$

In other words: Number of orderlines of the specific products over the total number of order lines of the dataset

SKU B's likelyhood to be bought when SKU A is bought:  $$ Lift(A\Rightarrow B) = \frac{Support}{Supp(A) * Supp(B)}) $$

The orders are so small (93% SLO) that I suspect the assiciation will be very small. As a reminder, the order profile:

In [ ]:
# sns.countplot(df_upo_lpo['lines']);

Only for orders with 2 or more lines association can be extracted.

I will extract these MLOs and perform with these the Affinity Analysis.

### 2. Preparing data

Only multi-line orders are interesting, because here we can observe which categories are ordered together.

In [ ]:
# List with all multi-line orders
#lst_mlo = df_upo_lpo[df_upo_lpo['lines']>1]['order_id'].tolist()

In [ ]:
# Extract dataframe with only multi-line orders
#df_mlo = df_orders_consolidated[df_orders_consolidated['order_id'].isin(lst_mlo)]

In [3]:
# Generate sparse matrix by using my matric function
#mat_mlo_cat = create_category_order_matrix(df_mlo, 'order_id', 'product_category_name_english', 'qty')

In [4]:
#mat_mlo_cat = mat_mlo_cat['qty']

### Support Score
The support score for the prodcuts in this table is simply the mean

In [ ]:
#mat_mlo_cat.mean().sort_values(ascending=False).head(20)

### 3. Performing Analysis

In [5]:
# Aryori needs a list with all mlos, where each lmo contains a list with all categories
#records = []
#for i in range(3229):
#    records.append([mat_mlo_cat.values[i, j] for j in range(0, 66)])

In [6]:
# Applying list to apyori algorithm
#affinity_model = apriori(mat_mlo_cat, min_support=0.01, min_confidence=0.2 , use_colnames = True)
#affinity_results = list(affinity_model)

# In Process ...